<a href="https://colab.research.google.com/github/prajeet26/NER-LLM/blob/main/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **QLoRa Fintuned Automative NER**

### imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.0 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

### Load model

In [44]:
import torch

torch.cuda.empty_cache()

In [45]:
model_name = "openaccess-ai-collective/tiny-mistral"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

In [47]:
!nvidia-smi

Tue Mar  5 22:44:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              26W /  70W |  11619MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Load tokenizer

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Using Base Model

#### Prompt Engineering

### Prepare Model for Training

In [27]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

In [28]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

### Preparing Training Dataset

In [29]:
import pandas as pd
data = pd.read_csv("/content/combined.csv")
data

,combined_text,entities
0,"DESCDEF: ON CERTAIN PULL, FIFTH WHEEL AND TOY-...","[{'Entity': '1-888-446-5157', 'Label': 'Contac..."
1,DESCDEF: E-TON IS RECALLING CERTAIN MODEL YEAR...,"[{'Entity': '1999', 'Label': 'Year'}, {'Entity..."
2,"DESCDEF: NAVISTAR IS RECALLING 51,588 HEAVY TR...","[{'Entity': '2000', 'Label': 'Year'}, {'Entity..."
3,"DESCDEF: Nissan North America, Inc. (Nissan) i...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity..."
4,"DESCDEF: Altec Industries, Inc. is recalling c...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity..."
5,"DESCDEF: KZRV, L.P. (KZRV) is recalling variou...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity..."
6,DESCDEF: ON CERTAIN BUMPER-PULL HORSE TRAILERS...,"[{'Entity': '2002', 'Label': 'Year'}, {'Entity..."
7,DESCDEF: ON CERTAIN PASSENGER VEHICLES EQUIPPE...,"[{'Entity': '2002', 'Label': 'Year'}, {'Entity..."
8,"DESCDEF: ON CERTAIN HEAVY DUTY CLASS 8 TRUCKS,...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity..."
9,DESCDEF: ON CERTAIN FIRE TRUCKS WITH TANDEM ST...,"[{'Entity': '2003', 'Label': 'Year'}, {'Entity..."


In [30]:
entlist = [
    "Manufacturer:like name of the car manufacturer",
    "Model: for example like  a model like focus from manufacturer ford",
    "Year: for example like made  in the  year 2000",
    "Recall_Number:for example like  02V288000",
    "Component: for example like ELECTRICAL SYSTEM or  BATTERY CABLES",
    "Issue_Defect: for example like CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC ENGINES or  LOOSE OR BROKEN ATTACHMENTS AND MISROUTED BATTERY CABLES COULD LEAD TO CABLE INSULATION DAMAGE",
    "Notification_Dates:for example like  OWNER NOTIFICATION BEGAN FEBRUARY 10, 2003.",
    "Corrective Action:for example like  DEALERS WILL INSPECT THE BATTERY CABLES FOR THE CONDITION OF THE CABLE INSULATION AND PROPER TIGHTENING OF THE TERMINAL ENDS or AS NECESSARY, CABLES WILL BE REROUTED, RETAINING CLIPS INSTALLED, AND DAMAGED BATTERY CABLES REPLACED.",
    "Contact_Information:for example like  1-866-436-7332. or SAFETY ADMINISTRATION'S AUTO SAFETY HOTLINE AT 1-888-DASH-2-DOT (1-888-327-4236).",
    "high risk: when really high risk causing heavy injury like brain damage/ribcage fracture/ legs broken etc ",
    "low risk:Low risk like small sprains/burns/bruises etc"
    "safety risk:airbag deployment failure, brake failure, acceleration issues, seatbelt malfunction, tire blowouts, electrical system failure, fuel system leaks, compromised structural integrity}"
]

instr ="You are a smart and intelligent Name Entity Recognition (NER) System. " \
           "The task involves using to extract and identify all instances of automotive domain entities " \
           "from a given dataset containing text related to the automotive domain. The identified entities should " \
           "be enclosed within JSON keys corresponding to their entity type in the output, which must have the same content as " \
           """the input. For instance, a car  manufacturer should be represented as [{"Entity": "", "Label": "Failure Issue"}"""\
           "while a car model should be represented as \"car_model\": \"<Model of a car>\". It is important " \
           "to ensure that all relevant automotive domain entities are identified, and false identifications are avoided." \
           f"""\n\nHere are the List Of Entities\n{entlist}"""\
           "\n\n" \
           "I would want you to identify all instances of automotive domain entities in the below text from in the format like this "\
           """[{"Entity": "bottoming out the suspension", "Label": "Failure Issue"},
              {"Entity": "amplification of the stress", "Label": "Failure Issue"},
              {"Entity": "floor truss network", "Label": "Component"},"""\
           "with reference to the enitity list already given there should be No false idenitifications, dont give me any process just idenitfy the entities properly with no mistakes"\
           "With Each entity-label pair in a newline. Are the instructions clear to you?"\


data["instruction"]=instr
data

,combined_text,entities,instruction
0,"DESCDEF: ON CERTAIN PULL, FIFTH WHEEL AND TOY-...","[{'Entity': '1-888-446-5157', 'Label': 'Contac...",You are a smart and intelligent Name Entity Re...
1,DESCDEF: E-TON IS RECALLING CERTAIN MODEL YEAR...,"[{'Entity': '1999', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
2,"DESCDEF: NAVISTAR IS RECALLING 51,588 HEAVY TR...","[{'Entity': '2000', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
3,"DESCDEF: Nissan North America, Inc. (Nissan) i...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
4,"DESCDEF: Altec Industries, Inc. is recalling c...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
5,"DESCDEF: KZRV, L.P. (KZRV) is recalling variou...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
6,DESCDEF: ON CERTAIN BUMPER-PULL HORSE TRAILERS...,"[{'Entity': '2002', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
7,DESCDEF: ON CERTAIN PASSENGER VEHICLES EQUIPPE...,"[{'Entity': '2002', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
8,"DESCDEF: ON CERTAIN HEAVY DUTY CLASS 8 TRUCKS,...","[{'Entity': '2002', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...
9,DESCDEF: ON CERTAIN FIRE TRUCKS WITH TANDEM ST...,"[{'Entity': '2003', 'Label': 'Year'}, {'Entity...",You are a smart and intelligent Name Entity Re...


In [31]:
print(len(instr))

2640


In [32]:
def formatting_func(example):
    text = f"### {example['instruction']}: \n### User: {example['combined_text']} \n### Assistant: {example['entities']}"
    return text

In [33]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data, test_size=0.2, random_state=42)


In [34]:
tokenizer.pad_token = tokenizer.eos_token

In [35]:
from datasets import Dataset
max_length = 2300 # differs from datasets to datasets
def generate_and_tokenize_prompt(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

train_dataset = Dataset.from_pandas(df_train)
eval_dataset = Dataset.from_pandas(df_test)

train_dataset = train_dataset
eval_dataset = eval_dataset
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

In [36]:

tokenizer.pad_token = tokenizer.eos_token

data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Fine-tuning Model

In [39]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1500"

In [48]:
import transformers
from datetime import datetime

project = "LLM-NER"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500,
        learning_rate=2.5e-4,
        optim="paged_adamw_8bit",
        logging_steps=25,
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=25,
        evaluation_strategy="steps",
        eval_steps=25,
        do_eval=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,5.333500,2.187877
50,1.601700,1.320358
75,0.970200,1.181173
100,0.801900,1.141954
125,0.695100,1.125628
150,0.522700,1.097551
175,0.549200,1.117222
200,0.468400,1.103067
225,0.349000,1.110708
250,0.285200,1.119694


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=500, training_loss=0.6241841201782227, metrics={'train_runtime': 1232.8046, 'train_samples_per_second': 0.406, 'train_steps_per_second': 0.406, 'total_flos': 1364794828800000.0, 'train_loss': 0.6241841201782227, 'epoch': 15.15})